![image.png](../background_photos/)
[լուսանկարի հղումը](https://unsplash.com/photos/a-large-mountain-with-a-very-tall-cliff-UiP9KfVe3aQ), Հեղինակ՝ []()

<a href="ToDo" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> (ToDo)

> Song reference - ToDo

# 📌 Նկարագիր

[📚 Ամբողջական նյութը]()

#### 📺 Տեսանյութեր
#### 🏡 Տնային

# 📚 Pydantic Tutorial

## What is Pydantic?

**Pydantic** is a Python library that provides data validation and settings management using Python type annotations. It's designed to make data validation simple, fast, and reliable.

### Key Features:
- **Type Safety**: Uses Python type hints for validation
- **Fast**: Written in Rust (v2) for performance
- **Easy to Use**: Intuitive API design
- **Comprehensive**: Supports complex data structures
- **Integration**: Works seamlessly with FastAPI

### Why Use Pydantic?
1. **Data Validation**: Automatically validates input data
2. **Type Conversion**: Converts compatible types automatically
3. **Error Handling**: Provides detailed error messages
4. **Documentation**: Auto-generates JSON schemas
5. **IDE Support**: Full type checking and autocomplete

## Installation

```bash
pip install pydantic
```

For email validation and other extras:
```bash
pip install pydantic[email]
```

In [ ]:
# Install required packages
# !pip install pydantic fastapi uvicorn

# Import necessary libraries
from pydantic import BaseModel, validator, ValidationError, Field
from typing import Optional, List, Dict, Union
from datetime import datetime
from enum import Enum
import json

print("Pydantic version:", __import__('pydantic').__version__)

## Basic Pydantic Models

A Pydantic model is a class that inherits from `BaseModel`. All attributes are defined with type annotations.

In [ ]:
# Basic Pydantic Model
class User(BaseModel):
    id: int
    name: str
    email: str
    age: Optional[int] = None
    is_active: bool = True

# Creating an instance
user_data = {
    "id": 1,
    "name": "John Doe",
    "email": "john@example.com",
    "age": 30
}

user = User(**user_data)
print("User created:")
print(user)
print(f"User name: {user.name}")
print(f"User email: {user.email}")

# Convert to dictionary
print("\nAs dictionary:")
print(user.model_dump())

# Convert to JSON
print("\nAs JSON:")
print(user.model_dump_json(indent=2))

## Data Validation

Pydantic automatically validates data types and provides helpful error messages.

In [ ]:
# Type conversion - Pydantic tries to convert compatible types
print("=== Type Conversion ===")
user_flexible = User(id="123", name="Jane", email="jane@example.com", age="25")
print(f"ID (converted from string): {user_flexible.id} (type: {type(user_flexible.id)})")
print(f"Age (converted from string): {user_flexible.age} (type: {type(user_flexible.age)})")

# Validation errors
print("\n=== Validation Errors ===")
try:
    invalid_user = User(id="not_a_number", name="Test", email="invalid_email")
except ValidationError as e:
    print("Validation failed:")
    for error in e.errors():
        print(f"  Field: {error['loc'][0]}")
        print(f"  Error: {error['msg']}")
        print(f"  Value: {error['input']}")
        print("---")

# Missing required fields
print("\n=== Missing Required Fields ===")
try:
    incomplete_user = User(name="Only Name")
except ValidationError as e:
    print("Missing fields:")
    for error in e.errors():
        print(f"  {error['loc'][0]}: {error['msg']}")

## Field Constraints and Custom Validators

Pydantic provides Field constraints and custom validators for more sophisticated validation.

In [ ]:
# Advanced model with Field constraints and custom validators
from pydantic import field_validator, model_validator
import re

class AdvancedUser(BaseModel):
    id: int = Field(gt=0, description="User ID must be positive")
    username: str = Field(min_length=3, max_length=20, description="Username between 3-20 characters")
    email: str = Field(description="Valid email address")
    age: Optional[int] = Field(default=None, ge=0, le=150, description="Age between 0-150")
    password: str = Field(min_length=8, description="Password minimum 8 characters")
    confirm_password: str = Field(description="Password confirmation")
    full_name: str = Field(description="Full name")
    
    @field_validator('email')
    @classmethod
    def validate_email(cls, v):
        # Simple email validation
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not re.match(pattern, v):
            raise ValueError('Invalid email format')
        return v
    
    @field_validator('username')
    @classmethod
    def validate_username(cls, v):
        if not v.isalnum():
            raise ValueError('Username must be alphanumeric')
        return v
    
    @field_validator('password')
    @classmethod
    def validate_password(cls, v):
        if not any(c.isupper() for c in v):
            raise ValueError('Password must contain at least one uppercase letter')
        if not any(c.islower() for c in v):
            raise ValueError('Password must contain at least one lowercase letter')
        if not any(c.isdigit() for c in v):
            raise ValueError('Password must contain at least one digit')
        return v
    
    @model_validator(mode='after')
    def validate_passwords_match(self):
        if self.password != self.confirm_password:
            raise ValueError('Passwords do not match')
        return self

# Test the advanced model
print("=== Valid User ===")
try:
    advanced_user = AdvancedUser(
        id=1,
        username="johndoe123",
        email="john@example.com",
        age=25,
        password="SecurePass123",
        confirm_password="SecurePass123",
        full_name="John Doe"
    )
    print("✅ User created successfully!")
    print(advanced_user.model_dump(exclude={'password', 'confirm_password'}))
except ValidationError as e:
    print("❌ Validation failed:")
    for error in e.errors():
        print(f"  {error['loc']}: {error['msg']}")

print("\n=== Invalid User (weak password) ===")
try:
    invalid_user = AdvancedUser(
        id=1,
        username="johndoe123",
        email="john@example.com",
        age=25,
        password="weak",  # Too weak
        confirm_password="weak",
        full_name="John Doe"
    )
except ValidationError as e:
    print("❌ Validation failed:")
    for error in e.errors():
        print(f"  {error['loc']}: {error['msg']}")

## Complex Data Structures and Nested Models

Pydantic can handle complex nested structures, lists, and enums.

In [ ]:
# Enums for better type safety
class UserRole(str, Enum):
    ADMIN = "admin"
    USER = "user"
    MODERATOR = "moderator"

class Status(str, Enum):
    ACTIVE = "active"
    INACTIVE = "inactive"
    PENDING = "pending"

# Nested models
class Address(BaseModel):
    street: str
    city: str
    country: str
    postal_code: str = Field(regex=r'^\d{5}(-\d{4})?$')  # US ZIP code format

class Contact(BaseModel):
    phone: Optional[str] = None
    emergency_contact: Optional[str] = None

class Profile(BaseModel):
    bio: Optional[str] = None
    avatar_url: Optional[str] = None
    social_links: Dict[str, str] = {}

# Complex user model with nested structures
class ComplexUser(BaseModel):
    id: int
    username: str
    email: str
    role: UserRole
    status: Status = Status.ACTIVE
    addresses: List[Address] = []
    contact: Optional[Contact] = None
    profile: Optional[Profile] = None
    tags: List[str] = []
    metadata: Dict[str, Union[str, int, bool]] = {}
    created_at: datetime = Field(default_factory=datetime.now)

# Example data
complex_user_data = {
    "id": 1,
    "username": "johndoe",
    "email": "john@example.com",
    "role": "admin",
    "addresses": [
        {
            "street": "123 Main St",
            "city": "New York",
            "country": "USA",
            "postal_code": "10001"
        },
        {
            "street": "456 Oak Ave",
            "city": "Los Angeles",
            "country": "USA",
            "postal_code": "90210"
        }
    ],
    "contact": {
        "phone": "+1-555-123-4567",
        "emergency_contact": "+1-555-987-6543"
    },
    "profile": {
        "bio": "Software developer and tech enthusiast",
        "avatar_url": "https://example.com/avatar.jpg",
        "social_links": {
            "twitter": "https://twitter.com/johndoe",
            "linkedin": "https://linkedin.com/in/johndoe"
        }
    },
    "tags": ["developer", "python", "ai"],
    "metadata": {
        "login_count": 42,
        "premium": True,
        "last_ip": "192.168.1.1"
    }
}

# Create complex user
complex_user = ComplexUser(**complex_user_data)
print("=== Complex User Created ===")
print(f"Username: {complex_user.username}")
print(f"Role: {complex_user.role}")
print(f"Number of addresses: {len(complex_user.addresses)}")
print(f"First address: {complex_user.addresses[0].street}, {complex_user.addresses[0].city}")
print(f"Phone: {complex_user.contact.phone}")
print(f"Bio: {complex_user.profile.bio}")
print(f"Tags: {', '.join(complex_user.tags)}")
print(f"Created at: {complex_user.created_at}")

# JSON output (with pretty formatting)
print("\n=== JSON Representation ===")
print(complex_user.model_dump_json(indent=2, exclude={'created_at'}))

## Pydantic vs. Python Dataclasses

Let's compare Pydantic with Python's built-in dataclasses to understand when to use each.

In [ ]:
# Comparison: Pydantic vs Dataclasses
from dataclasses import dataclass, field
from typing import List, Optional
import json

print("=== PYDANTIC vs DATACLASSES COMPARISON ===\n")

# ==========================================
# 1. BASIC STRUCTURE COMPARISON
# ==========================================

# Python Dataclass
@dataclass
class PersonDataclass:
    name: str
    age: int
    email: str
    is_active: bool = True

# Pydantic Model
class PersonPydantic(BaseModel):
    name: str
    age: int
    email: str
    is_active: bool = True

print("1. BASIC STRUCTURE:")
print("Both support type hints and default values")

# Create instances
dc_person = PersonDataclass("John", 30, "john@example.com")
pyd_person = PersonPydantic(name="John", age=30, email="john@example.com")

print(f"Dataclass: {dc_person}")
print(f"Pydantic: {pyd_person}")

# ==========================================
# 2. TYPE VALIDATION AND CONVERSION
# ==========================================

print("\n2. TYPE VALIDATION AND CONVERSION:")

# Dataclasses - NO automatic validation
print("\nDataclasses (NO validation):")
try:
    dc_invalid = PersonDataclass("John", "thirty", "invalid-email")  # Wrong types
    print(f"✅ Created successfully: {dc_invalid}")
    print(f"Age type: {type(dc_invalid.age)} (should be int!)")
except Exception as e:
    print(f"❌ Error: {e}")

# Pydantic - Automatic validation and conversion
print("\nPydantic (WITH validation):")
try:
    pyd_converted = PersonPydantic(name="John", age="30", email="john@example.com")  # String age
    print(f"✅ Created with type conversion: {pyd_converted}")
    print(f"Age type: {type(pyd_converted.age)} (converted from string!)")
except ValidationError as e:
    print(f"❌ Validation error: {e}")

try:
    pyd_invalid = PersonPydantic(name="John", age="not_a_number", email="john@example.com")
except ValidationError as e:
    print(f"❌ Pydantic caught invalid data: {e.errors()[0]['msg']}")

# ==========================================
# 3. JSON SERIALIZATION
# ==========================================

print("\n3. JSON SERIALIZATION:")

# Dataclass - needs extra work
print("\nDataclass JSON serialization:")
try:
    # This will fail because dataclasses don't have built-in JSON support
    dc_json = json.dumps(dc_person)
    print("❌ This won't work")
except TypeError as e:
    print(f"❌ Error: {e}")

# Need to convert to dict first
dc_dict = {"name": dc_person.name, "age": dc_person.age, "email": dc_person.email, "is_active": dc_person.is_active}
dc_json = json.dumps(dc_dict)
print(f"✅ Manual conversion: {dc_json}")

# Pydantic - built-in JSON support
print("\nPydantic JSON serialization:")
pyd_json = pyd_person.model_dump_json()
print(f"✅ Built-in method: {pyd_json}")

# ==========================================
# 4. COMPLEX VALIDATION
# ==========================================

print("\n4. COMPLEX VALIDATION:")

# Dataclass with manual validation
@dataclass
class UserDataclass:
    username: str
    email: str
    age: int
    
    def __post_init__(self):
        # Manual validation (tedious!)
        if len(self.username) < 3:
            raise ValueError("Username too short")
        if "@" not in self.email:
            raise ValueError("Invalid email")
        if self.age < 0:
            raise ValueError("Age cannot be negative")

# Pydantic with built-in validation
class UserPydantic(BaseModel):
    username: str = Field(min_length=3)
    email: str = Field(regex=r'^[^@]+@[^@]+\.[^@]+$')
    age: int = Field(ge=0)

print("\nDataclass validation (manual __post_init__):")
try:
    dc_user = UserDataclass("ab", "invalid", -5)
except ValueError as e:
    print(f"❌ Manual validation caught: {e}")

print("\nPydantic validation (declarative):")
try:
    pyd_user = UserPydantic(username="ab", email="invalid", age=-5)
except ValidationError as e:
    errors = e.errors()
    for error in errors:
        print(f"❌ {error['loc'][0]}: {error['msg']}")

# ==========================================
# 5. NESTED STRUCTURES
# ==========================================

print("\n5. NESTED STRUCTURES:")

@dataclass
class AddressDataclass:
    street: str
    city: str

@dataclass
class UserWithAddressDataclass:
    name: str
    address: AddressDataclass

class AddressPydantic(BaseModel):
    street: str
    city: str

class UserWithAddressPydantic(BaseModel):
    name: str
    address: AddressPydantic

# Dataclass - manual creation
dc_address = AddressDataclass("123 Main St", "NYC")
dc_user_with_addr = UserWithAddressDataclass("John", dc_address)

# Pydantic - automatic nested parsing from dict
pyd_user_with_addr = UserWithAddressPydantic(
    name="John",
    address={"street": "123 Main St", "city": "NYC"}  # Dict automatically converted!
)

print(f"Dataclass nested: {dc_user_with_addr}")
print(f"Pydantic nested: {pyd_user_with_addr}")

# ==========================================
# 6. PERFORMANCE COMPARISON
# ==========================================

print("\n6. PERFORMANCE COMPARISON:")
import time

def time_creation(creation_func, iterations=10000):
    start = time.time()
    for _ in range(iterations):
        creation_func()
    return time.time() - start

def create_dataclass():
    return PersonDataclass("Test", 25, "test@example.com")

def create_pydantic():
    return PersonPydantic(name="Test", age=25, email="test@example.com")

dc_time = time_creation(create_dataclass, 1000)
pyd_time = time_creation(create_pydantic, 1000)

print(f"Dataclass creation (1000x): {dc_time:.4f}s")
print(f"Pydantic creation (1000x): {pyd_time:.4f}s")
print(f"Pydantic is {pyd_time/dc_time:.1f}x slower (due to validation)")

# ==========================================
# 7. SUMMARY TABLE
# ==========================================

print("\n" + "="*50)
print("FEATURE COMPARISON SUMMARY:")
print("="*50)

comparison_data = [
    ["Feature", "Dataclass", "Pydantic"],
    ["-" * 20, "-" * 15, "-" * 15],
    ["Type Hints", "✅ Yes", "✅ Yes"],
    ["Runtime Validation", "❌ No", "✅ Yes"],
    ["Type Conversion", "❌ No", "✅ Yes"],
    ["JSON Serialization", "❌ Manual", "✅ Built-in"],
    ["Custom Validation", "❌ Manual", "✅ Declarative"],
    ["Error Messages", "❌ Basic", "✅ Detailed"],
    ["Performance", "✅ Fast", "⚠️ Slower"],
    ["Memory Usage", "✅ Low", "⚠️ Higher"],
    ["Learning Curve", "✅ Easy", "⚠️ Medium"],
    ["API Integration", "❌ Manual work", "✅ Perfect"],
    ["Schema Generation", "❌ No", "✅ Yes"],
]

for row in comparison_data:
    print(f"{row[0]:<20} {row[1]:<15} {row[2]:<15}")

print("\n" + "="*50)
print("WHEN TO USE EACH:")
print("="*50)
print("🏗️  DATACLASSES:")
print("   • Simple data containers")
print("   • Performance-critical applications")
print("   • Internal data structures")
print("   • When you trust your data sources")
print("   • Standard library only")

print("\n🔒 PYDANTIC:")
print("   • APIs and external data")
print("   • Data validation is critical")
print("   • JSON serialization needed")
print("   • FastAPI integration")
print("   • Configuration management")
print("   • Data parsing from untrusted sources")

# 🚀 FastAPI Tutorial

## What is FastAPI?

**FastAPI** is a modern, fast (high-performance) web framework for building APIs with Python 3.7+ based on standard Python type hints.

### Key Features:
- **Fast**: One of the fastest Python frameworks available
- **Easy**: Designed to be easy to use and learn
- **Standards-based**: Based on OpenAPI and JSON Schema
- **Automatic docs**: Interactive API documentation
- **Type Safety**: Full type checking with Pydantic
- **Modern**: Uses async/await for high performance

### Why FastAPI?
1. **Performance**: Comparable to NodeJS and Go
2. **Developer Experience**: Great autocomplete and error detection
3. **Standards**: Follows web standards (OpenAPI, JSON Schema)
4. **Documentation**: Automatic interactive docs
5. **Production Ready**: Used by major companies

## Understanding REST APIs

### What is REST?

**REST** (Representational State Transfer) is an architectural style for designing networked applications. A RESTful API follows REST principles.

#### REST Principles:
1. **Stateless**: Each request contains all information needed
2. **Client-Server**: Separation of concerns
3. **Cacheable**: Responses should be cacheable when possible
4. **Uniform Interface**: Consistent way to interact with resources
5. **Layered System**: Architecture can have multiple layers

#### HTTP Methods (Verbs):
- **GET**: Retrieve data (Read)
- **POST**: Create new resources (Create)
- **PUT**: Update entire resource (Update/Replace)
- **PATCH**: Partial update (Modify)
- **DELETE**: Remove resource (Delete)

#### HTTP Status Codes:
- **200**: OK - Success
- **201**: Created - Resource created successfully
- **400**: Bad Request - Invalid request
- **401**: Unauthorized - Authentication required
- **403**: Forbidden - Access denied
- **404**: Not Found - Resource doesn't exist
- **422**: Unprocessable Entity - Validation error
- **500**: Internal Server Error - Server error

### REST API Structure
```
GET    /users           # Get all users
GET    /users/123       # Get user with ID 123
POST   /users           # Create new user
PUT    /users/123       # Update user 123 (full update)
PATCH  /users/123       # Partially update user 123
DELETE /users/123       # Delete user 123
```

In [ ]:
# FastAPI Installation and Setup
# !pip install fastapi uvicorn

from fastapi import FastAPI, HTTPException, Query, Path, Body, Depends
from fastapi.responses import HTMLResponse
from typing import List, Optional
import uvicorn

# Create FastAPI instance
app = FastAPI(
    title="My API",
    description="A sample API built with FastAPI",
    version="1.0.0"
)

print("FastAPI app created!")

## Basic FastAPI Routes

Let's create some basic API endpoints to understand how FastAPI works.

In [ ]:
# Basic Routes

# Root endpoint
@app.get("/")
async def root():
    return {"message": "Hello World"}

# Simple GET endpoint
@app.get("/hello/{name}")
async def say_hello(name: str):
    return {"message": f"Hello {name}!"}

# GET with query parameters
@app.get("/items/")
async def read_items(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

# Multiple path parameters
@app.get("/users/{user_id}/items/{item_id}")
async def read_user_item(user_id: int, item_id: str):
    return {"user_id": user_id, "item_id": item_id}

# Optional query parameters
@app.get("/search/")
async def search_items(q: Optional[str] = None, category: str = "all"):
    if q:
        return {"query": q, "category": category, "results": f"Results for '{q}' in {category}"}
    return {"message": "No query provided", "category": category}

print("Basic routes defined!")

## Using Pydantic Models with FastAPI

FastAPI integrates seamlessly with Pydantic for request/response validation.

In [ ]:
# Pydantic models for API
class UserCreate(BaseModel):
    username: str = Field(min_length=3, max_length=20)
    email: str = Field(description="Valid email address")
    full_name: str
    age: Optional[int] = Field(default=None, ge=0, le=150)

class UserResponse(BaseModel):
    id: int
    username: str
    email: str
    full_name: str
    age: Optional[int]
    is_active: bool = True
    created_at: datetime

class UserUpdate(BaseModel):
    username: Optional[str] = Field(default=None, min_length=3, max_length=20)
    email: Optional[str] = None
    full_name: Optional[str] = None
    age: Optional[int] = Field(default=None, ge=0, le=150)
    is_active: Optional[bool] = None

# In-memory database (for demo purposes)
fake_users_db = {}
user_id_counter = 1

# API endpoints with Pydantic models

@app.post("/users/", response_model=UserResponse)
async def create_user(user: UserCreate):
    global user_id_counter
    
    # Check if username already exists
    for existing_user in fake_users_db.values():
        if existing_user['username'] == user.username:
            raise HTTPException(status_code=400, detail="Username already exists")
    
    # Create new user
    new_user = {
        "id": user_id_counter,
        "username": user.username,
        "email": user.email,
        "full_name": user.full_name,
        "age": user.age,
        "is_active": True,
        "created_at": datetime.now()
    }
    
    fake_users_db[user_id_counter] = new_user
    user_id_counter += 1
    
    return new_user

@app.get("/users/", response_model=List[UserResponse])
async def get_users(skip: int = 0, limit: int = 10):
    users = list(fake_users_db.values())
    return users[skip:skip + limit]

@app.get("/users/{user_id}", response_model=UserResponse)
async def get_user(user_id: int = Path(description="The ID of the user to get")):
    if user_id not in fake_users_db:
        raise HTTPException(status_code=404, detail="User not found")
    return fake_users_db[user_id]

@app.put("/users/{user_id}", response_model=UserResponse)
async def update_user(user_id: int, user_update: UserUpdate):
    if user_id not in fake_users_db:
        raise HTTPException(status_code=404, detail="User not found")
    
    stored_user = fake_users_db[user_id]
    update_data = user_update.model_dump(exclude_unset=True)
    
    for field, value in update_data.items():
        stored_user[field] = value
    
    return stored_user

@app.delete("/users/{user_id}")
async def delete_user(user_id: int):
    if user_id not in fake_users_db:
        raise HTTPException(status_code=404, detail="User not found")
    
    deleted_user = fake_users_db.pop(user_id)
    return {"message": f"User {deleted_user['username']} deleted successfully"}

print("User API endpoints defined!")

## Advanced FastAPI Features

### Query Parameters with Validation
### Dependencies
### Error Handling
### Background Tasks

In [ ]:
# Advanced Query Parameters with Validation
@app.get("/advanced-search/")
async def advanced_search(
    q: str = Query(description="Search query", min_length=1, max_length=50),
    category: str = Query(default="all", regex="^(all|books|electronics|clothing)$"),
    min_price: float = Query(default=0, ge=0, description="Minimum price"),
    max_price: float = Query(default=1000, le=10000, description="Maximum price"),
    sort_by: str = Query(default="relevance", regex="^(relevance|price|date)$"),
    page: int = Query(default=1, ge=1, le=100, description="Page number")
):
    return {
        "query": q,
        "category": category,
        "price_range": {"min": min_price, "max": max_price},
        "sort_by": sort_by,
        "page": page,
        "results": f"Found results for '{q}' in {category}"
    }

# Dependencies
async def get_current_user_id(user_id: int = Query(description="Current user ID")):
    # In a real app, this would validate JWT token
    if user_id <= 0:
        raise HTTPException(status_code=400, detail="Invalid user ID")
    return user_id

async def get_pagination_params(skip: int = Query(default=0, ge=0), limit: int = Query(default=10, ge=1, le=100)):
    return {"skip": skip, "limit": limit}

@app.get("/my-profile/")
async def get_my_profile(current_user_id: int = Depends(get_current_user_id)):
    if current_user_id not in fake_users_db:
        raise HTTPException(status_code=404, detail="User not found")
    return fake_users_db[current_user_id]

@app.get("/users-paginated/", response_model=List[UserResponse])
async def get_users_paginated(
    pagination: dict = Depends(get_pagination_params),
    current_user_id: int = Depends(get_current_user_id)
):
    users = list(fake_users_db.values())
    skip = pagination["skip"]
    limit = pagination["limit"]
    return users[skip:skip + limit]

# Custom Exception Handler
class CustomException(Exception):
    def __init__(self, message: str):
        self.message = message

@app.exception_handler(CustomException)
async def custom_exception_handler(request, exc: CustomException):
    return {"error": "Custom Error", "message": exc.message, "status_code": 400}

@app.get("/trigger-error/")
async def trigger_error():
    raise CustomException("This is a custom error!")

# Background Tasks
from fastapi import BackgroundTasks
import time

def write_log(message: str):
    # Simulate time-consuming task
    time.sleep(2)
    print(f"LOG: {message} - {datetime.now()}")

@app.post("/send-notification/")
async def send_notification(email: str, background_tasks: BackgroundTasks):
    background_tasks.add_task(write_log, f"Notification sent to {email}")
    return {"message": "Notification will be sent in the background"}

# Health check endpoint
@app.get("/health/")
async def health_check():
    return {
        "status": "healthy",
        "timestamp": datetime.now(),
        "version": "1.0.0",
        "users_count": len(fake_users_db)
    }

print("Advanced FastAPI features added!")

## Running the FastAPI Server

To run your FastAPI application, you have several options:

In [ ]:
# Running FastAPI Server

# Option 1: Run with uvicorn directly (uncomment to run)
# if __name__ == "__main__":
#     uvicorn.run(app, host="0.0.0.0", port=8000)

# Option 2: Save to a file (e.g., main.py) and run from terminal:
# uvicorn main:app --reload

# Option 3: Run in development mode with auto-reload
# uvicorn main:app --reload --host 0.0.0.0 --port 8000

# Get the OpenAPI schema
print("=== OpenAPI Schema Info ===")
print(f"Title: {app.title}")
print(f"Version: {app.version}")
print(f"Description: {app.description}")

# List all routes
print("\n=== Available Routes ===")
for route in app.routes:
    if hasattr(route, 'methods') and hasattr(route, 'path'):
        methods = ', '.join(route.methods)
        print(f"{methods}: {route.path}")

print("\n🌐 Once running, visit:")
print("📖 Interactive docs: http://localhost:8000/docs")
print("📋 Alternative docs: http://localhost:8000/redoc")
print("🔧 OpenAPI schema: http://localhost:8000/openapi.json")

# Example of testing endpoints (you would normally use requests library)
print("\n=== Example API Usage ===")
print("# Create a user")
print("POST /users/")
print("Body: {")
print('  "username": "johndoe",')
print('  "email": "john@example.com",')
print('  "full_name": "John Doe",')
print('  "age": 30')
print("}")

print("\n# Get all users")
print("GET /users/")

print("\n# Get specific user")
print("GET /users/1")

print("\n# Update user")
print("PUT /users/1")
print("Body: {")
print('  "full_name": "John Smith"')
print("}")

print("\n# Delete user")
print("DELETE /users/1")

## FastAPI vs Flask Comparison

Let's compare FastAPI with Flask, the most popular Python web framework, to understand their differences and use cases.

In [ ]:
### 1. Basic API Structure Comparison

#### Flask Version:
```python
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/')
def hello():
    return {"message": "Hello World"}

@app.route('/users/<int:user_id>')
def get_user(user_id):
    return {"user_id": user_id}

@app.route('/users', methods=['POST'])
def create_user():
    data = request.get_json()
    # Manual validation required
    if not data or 'name' not in data:
        return {"error": "Name is required"}, 400
    return {"message": f"User {data['name']} created"}, 201

if __name__ == '__main__':
    app.run(debug=True)
```

#### FastAPI Version:
```python
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class UserCreate(BaseModel):
    name: str

@app.get("/")
async def hello():
    return {"message": "Hello World"}

@app.get("/users/{user_id}")
async def get_user(user_id: int):
    return {"user_id": user_id}

@app.post("/users/")
async def create_user(user: UserCreate):
    # Automatic validation with Pydantic
    return {"message": f"User {user.name} created"}

# Run with: uvicorn main:app --reload
```

In [ ]:
### 2. Request Validation Comparison

**Flask**: Manual validation (tedious and error-prone)  
**FastAPI**: Automatic validation with Pydantic (declarative and robust)

In [ ]:
# Demonstration of validation differences
print("=== VALIDATION COMPARISON DEMO ===\n")

# Simulate Flask-style manual validation
def flask_style_validation():
    print("🌶️ FLASK - Manual Validation:")
    
    # Simulating request data
    request_data = {"name": "John", "age": "not_a_number", "email": "invalid_email"}
    
    errors = []
    
    # Manual validation (tedious!)
    if 'name' not in request_data:
        errors.append("Name is required")
    elif len(request_data['name']) < 2:
        errors.append("Name too short")
    
    if 'age' in request_data:
        try:
            age = int(request_data['age'])
            if age < 0:
                errors.append("Age cannot be negative")
        except ValueError:
            errors.append("Age must be a number")
    
    if 'email' in request_data:
        if '@' not in request_data['email']:
            errors.append("Invalid email format")
    
    if errors:
        print(f"❌ Validation errors: {errors}")
        return False
    
    print("✅ Validation passed")
    return True

# FastAPI automatic validation
class UserValidation(BaseModel):
    name: str = Field(min_length=2)
    age: int = Field(ge=0)
    email: str = Field(regex=r'^[^@]+@[^@]+\.[^@]+$')

def fastapi_style_validation():
    print("\n⚡ FASTAPI - Automatic Validation:")
    
    request_data = {"name": "John", "age": "not_a_number", "email": "invalid_email"}
    
    try:
        user = UserValidation(**request_data)
        print("✅ Validation passed")
        return True
    except ValidationError as e:
        print(f"❌ Validation errors:")
        for error in e.errors():
            print(f"   {error['loc'][0]}: {error['msg']}")
        return False

flask_style_validation()
fastapi_style_validation()

In [ ]:
### 3. Async Support Comparison

**Flask**: Synchronous by default (blocking operations)  
**FastAPI**: Native async/await support (concurrent processing)

In [ ]:
# Async Support Demonstration
import asyncio
import time

print("=== ASYNC SUPPORT DEMO ===\n")

# Flask - Synchronous (blocking)
def flask_sync_endpoint():
    print("🌶️ FLASK - Synchronous:")
    start = time.time()
    
    # Simulate database call
    time.sleep(0.1)  # Blocking call
    
    end = time.time()
    print(f"✅ Request completed in {end - start:.3f}s")
    return {"data": "result"}

# FastAPI - Asynchronous (non-blocking)
async def fastapi_async_endpoint():
    print("\n⚡ FASTAPI - Asynchronous:")
    start = time.time()
    
    # Simulate async database call
    await asyncio.sleep(0.1)  # Non-blocking call
    
    end = time.time()
    print(f"✅ Request completed in {end - start:.3f}s")
    return {"data": "result"}

# Demonstrate the difference
print("Single request timing:")
flask_sync_endpoint()
asyncio.run(fastapi_async_endpoint())

print("\nConcurrency simulation (5 requests):")

# Flask-style sequential processing
def simulate_flask_concurrent():
    start = time.time()
    for i in range(5):
        time.sleep(0.1)  # Each request blocks
    end = time.time()
    print(f"🌶️ Flask (sequential): {end - start:.3f}s for 5 requests")

# FastAPI-style concurrent processing
async def simulate_fastapi_concurrent():
    start = time.time()
    tasks = [asyncio.sleep(0.1) for _ in range(5)]
    await asyncio.gather(*tasks)  # All requests processed concurrently
    end = time.time()
    print(f"⚡ FastAPI (concurrent): {end - start:.3f}s for 5 requests")

simulate_flask_concurrent()
asyncio.run(simulate_fastapi_concurrent())

In [ ]:
### 4. Documentation and Type Hints Comparison

#### Documentation:

| Feature | Flask | FastAPI |
|---------|-------|---------|
| Auto Documentation | ❌ No | ✅ Built-in |
| API Schema | ❌ Manual | ✅ OpenAPI |
| Interactive Docs | ⚠️ Extensions | ✅ Swagger UI, ReDoc |
| Schema Validation | ❌ No | ✅ Yes |

#### Type Hints and IDE Support:

| Feature | Flask | FastAPI |
|---------|-------|---------|
| Type Hint Support | ⚠️ Limited | ✅ Full Support |
| Path Parameters | ❌ Always strings | ✅ Auto conversion |
| Request Validation | ❌ Manual | ✅ Automatic |
| IDE Autocomplete | ⚠️ Basic | ✅ Excellent |

**FastAPI Documentation URLs:**
- 📖 Interactive docs: `/docs`
- 📋 Alternative docs: `/redoc`
- 🔧 OpenAPI schema: `/openapi.json`

In [ ]:
### 5. Performance Comparison

#### Typical Benchmarks:
- **Flask**: ~30,000 requests/second
- **FastAPI**: ~60,000+ requests/second (with async)

> **Note**: FastAPI often performs better due to async support and optimized internals

In [ ]:
# Performance Testing Demo
import time

def performance_test():
    print("=== PERFORMANCE COMPARISON DEMO ===\n")
    
    # Simulate request processing
    iterations = 1000
    
    # Flask-style processing (synchronous)
    start = time.time()
    for _ in range(iterations):
        # Simulate request handling overhead
        data = {"name": "test", "age": 25}
        # Manual validation would add overhead
        pass
    flask_time = time.time() - start
    
    # FastAPI-style processing (with validation)
    class TestModel(BaseModel):
        name: str
        age: int
    
    start = time.time()
    for _ in range(iterations):
        # Automatic validation and serialization
        data = {"name": "test", "age": 25}
        TestModel(**data)
    fastapi_time = time.time() - start
    
    print(f"🌶️ Flask-style ({iterations} requests): {flask_time:.4f}s")
    print(f"⚡ FastAPI-style ({iterations} requests): {fastapi_time:.4f}s")
    
    if fastapi_time > flask_time:
        print(f"📊 FastAPI overhead: {((fastapi_time / flask_time - 1) * 100):.1f}% (due to validation)")
    else:
        print(f"📊 FastAPI is {((flask_time / fastapi_time - 1) * 100):.1f}% faster")

performance_test()

In [ ]:
### 6. Ecosystem and Extensions Comparison

| Feature | Flask | FastAPI |
|---------|-------|---------|
| **Template Engine** | Jinja2 (built-in) | Jinja2 (optional) |
| **Database ORM** | SQLAlchemy (Flask-SQLAlchemy) | SQLAlchemy, Tortoise ORM |
| **Authentication** | Flask-Login, Flask-JWT-Extended | Built-in OAuth2/JWT support |
| **API Documentation** | Flask-RESTX, Flask-Swagger | Built-in (Swagger UI, ReDoc) |
| **Validation** | WTForms, Marshmallow | Pydantic (built-in) |
| **Testing** | pytest, Flask's test client | pytest, TestClient (built-in) |
| **Admin Interface** | Flask-Admin | FastAPI-Admin, SQLAdmin |
| **Migration** | Flask-Migrate | Alembic |
| **Caching** | Flask-Caching | aiocache, Redis |
| **Background Tasks** | Celery | Built-in background tasks, Celery |

In [ ]:
### 7. Use Cases and Learning Curve

#### 🌶️ Flask Best For:
- 🌐 Traditional web applications with templates
- 📊 Admin dashboards and internal tools
- 🔧 Microservices with simple requirements
- 📚 Educational projects and prototypes
- 🎨 Applications requiring fine-grained control
- 💼 Existing codebases with Flask integration

#### ⚡ FastAPI Best For:
- 🚀 Modern REST APIs and microservices
- 📱 Mobile app backends
- 🤖 Machine learning model serving
- 🔗 API-first applications
- ⚡ High-performance real-time applications
- 📈 Data science and analytics APIs
- 🔄 Integration APIs and webhooks

#### Learning and Development Comparison:

| Aspect | Flask | FastAPI |
|--------|-------|---------|
| **Beginner Friendliness** | ✅ Very Easy | ⚠️ Medium |
| **Documentation Quality** | ✅ Excellent | ✅ Excellent |
| **Community Size** | ✅ Large | ⚠️ Growing |
| **Job Market** | ✅ Many opportunities | 📈 Rapidly growing |
| **Development Speed** | ⚠️ Slower (more setup) | ✅ Faster (less boilerplate) |
| **Debugging** | ✅ Mature tools | ✅ Good tools |
| **Type Safety** | ❌ Limited | ✅ Excellent |
| **Modern Features** | ⚠️ Requires extensions | ✅ Built-in |

In [ ]:
### 8. Comprehensive Feature Comparison

| Feature | Flask | FastAPI |
|---------|-------|---------|
| **Released** | 2010 | 2018 |
| **Type Hints** | ⚠️ Limited | ✅ Full Support |
| **Async/Await** | ⚠️ Extension | ✅ Native |
| **Auto Documentation** | ❌ No | ✅ Built-in |
| **Request Validation** | ❌ Manual | ✅ Automatic |
| **Response Serialization** | ❌ Manual | ✅ Automatic |
| **Performance** | ⚠️ Good | ✅ Excellent |
| **Learning Curve** | ✅ Easy | ⚠️ Medium |
| **Community** | ✅ Large | 📈 Growing |
| **Flexibility** | ✅ High | ⚠️ Opinionated |
| **Boilerplate** | ⚠️ More | ✅ Less |
| **Testing** | ✅ Mature | ✅ Excellent |
| **Deployment** | ✅ Many options | ✅ Modern options |
| **WebSocket Support** | ⚠️ Extension | ✅ Built-in |
| **Background Tasks** | ⚠️ External | ✅ Built-in |
| **Dependency Injection** | ❌ Manual | ✅ Built-in |
| **OpenAPI Schema** | ⚠️ Extension | ✅ Automatic |
| **Production Ready** | ✅ Very Mature | ✅ Production Ready |

In [ ]:
### 9. Migration Considerations and Final Recommendations

#### 🔄 Flask to FastAPI Migration:
- ✅ Similar routing concepts
- ✅ Can reuse business logic
- ⚠️ Need to add Pydantic models
- ⚠️ Convert to async (optional but recommended)
- ⚠️ Update dependencies and middleware
- ✅ Testing patterns are similar

#### 📈 FastAPI to Flask Migration:
- ⚠️ Lose automatic validation
- ⚠️ Need manual documentation
- ⚠️ More boilerplate code required
- ✅ More flexibility in some areas
- ⚠️ May need additional extensions

---

## 🎯 Final Recommendations

### 🌶️ Choose Flask When:
- Building traditional web applications
- Need maximum flexibility and control
- Team is new to web development
- Working with legacy systems
- Building simple, small applications
- Fine-grained customization is required

### ⚡ Choose FastAPI When:
- Building modern APIs
- Type safety is important
- Need automatic documentation
- Performance is critical
- Building microservices
- Team values modern Python features
- Integrating with ML/AI models

---

## 🎯 Conclusion

**Both frameworks are excellent choices!**

- **Flask**: Mature, flexible, great for learning
- **FastAPI**: Modern, fast, API-focused

**Choice depends on project requirements and team preferences**

## Testing Your API

FastAPI makes it easy to test your API endpoints.

In [ ]:
# Testing FastAPI with TestClient
from fastapi.testclient import TestClient

# Create a test client
client = TestClient(app)

def test_api_endpoints():
    print("=== Testing API Endpoints ===")
    
    # Test root endpoint
    response = client.get("/")
    print(f"GET /: {response.status_code} - {response.json()}")
    
    # Test hello endpoint
    response = client.get("/hello/FastAPI")
    print(f"GET /hello/FastAPI: {response.status_code} - {response.json()}")
    
    # Test search with query parameters
    response = client.get("/search/?q=python&category=books")
    print(f"GET /search/: {response.status_code} - {response.json()}")
    
    # Test creating a user
    user_data = {
        "username": "testuser",
        "email": "test@example.com",
        "full_name": "Test User",
        "age": 25
    }
    response = client.post("/users/", json=user_data)
    print(f"POST /users/: {response.status_code} - {response.json()}")
    
    if response.status_code == 200:
        user_id = response.json()["id"]
        
        # Test getting the user
        response = client.get(f"/users/{user_id}")
        print(f"GET /users/{user_id}: {response.status_code} - {response.json()}")
        
        # Test updating the user
        update_data = {"full_name": "Updated Test User"}
        response = client.put(f"/users/{user_id}", json=update_data)
        print(f"PUT /users/{user_id}: {response.status_code} - {response.json()}")
        
        # Test getting all users
        response = client.get("/users/")
        print(f"GET /users/: {response.status_code} - Found {len(response.json())} users")
    
    # Test error handling
    response = client.get("/users/999")  # Non-existent user
    print(f"GET /users/999 (not found): {response.status_code} - {response.json()}")
    
    # Test validation error
    invalid_user = {"username": "ab", "email": "invalid"}  # Too short username, invalid email
    response = client.post("/users/", json=invalid_user)
    print(f"POST /users/ (invalid): {response.status_code} - {response.json()}")

# Run the tests
test_api_endpoints()

## Best Practices

### Pydantic Best Practices:
1. **Use Field constraints** for validation
2. **Create separate models** for requests and responses
3. **Use custom validators** for complex validation logic
4. **Leverage type hints** for better IDE support
5. **Document your models** with descriptions

### FastAPI Best Practices:
1. **Use dependency injection** for reusable logic
2. **Separate concerns** - models, routes, business logic
3. **Handle errors gracefully** with proper HTTP status codes
4. **Use background tasks** for time-consuming operations
5. **Add proper documentation** to your endpoints
6. **Use environment variables** for configuration
7. **Implement proper authentication** and authorization
8. **Add logging** for debugging and monitoring

### Project Structure:
```
my_fastapi_project/
├── app/
│   ├── __init__.py
│   ├── main.py          # FastAPI app
│   ├── models/          # Pydantic models
│   │   ├── __init__.py
│   │   └── user.py
│   ├── routers/         # API routes
│   │   ├── __init__.py
│   │   └── users.py
│   ├── dependencies/    # Dependency functions
│   │   ├── __init__.py
│   │   └── auth.py
│   └── database/        # Database connection
│       ├── __init__.py
│       └── connection.py
├── tests/               # Test files
├── requirements.txt     # Dependencies
└── README.md           # Documentation
```

## Summary

### What We Learned:

#### Pydantic:
- ✅ Data validation with type hints
- ✅ Field constraints and custom validators
- ✅ Nested models and complex structures
- ✅ Automatic type conversion
- ✅ Detailed error messages

#### FastAPI:
- ✅ REST API principles and HTTP methods
- ✅ Creating API endpoints with different HTTP methods
- ✅ Request/response validation with Pydantic
- ✅ Query parameters and path parameters
- ✅ Dependency injection
- ✅ Error handling and custom exceptions
- ✅ Background tasks
- ✅ Automatic API documentation
- ✅ Testing with TestClient

### Next Steps:
1. **Database Integration** - Use SQLAlchemy or other ORMs
2. **Authentication** - Implement JWT or OAuth2
3. **Middleware** - Add CORS, logging, rate limiting
4. **Deployment** - Deploy to production with Docker
5. **Monitoring** - Add health checks and metrics
6. **Security** - Input sanitization, rate limiting

# 🛠️ Գործնական

## Exercise 1: Pydantic Models
Create a `Book` model with the following fields:
- `id`: integer (positive)
- `title`: string (3-100 characters)
- `author`: string (2-50 characters)  
- `isbn`: string (must match ISBN-10 or ISBN-13 format)
- `price`: float (positive)
- `publication_year`: integer (1900-2024)
- `genre`: enum (fiction, non-fiction, science, history, biography)
- `available`: boolean (default True)

Add custom validators to ensure:
- ISBN format is correct
- Publication year is not in the future
- Title and author don't contain only numbers

## Exercise 2: FastAPI Blog API
Create a simple blog API with the following endpoints:

### Models needed:
- `Post` (id, title, content, author_id, created_at, published)
- `Author` (id, name, email, bio)
- `Comment` (id, post_id, author_name, content, created_at)

### Endpoints to implement:
- `GET /posts/` - Get all posts (with pagination)
- `GET /posts/{post_id}` - Get specific post
- `POST /posts/` - Create new post
- `PUT /posts/{post_id}` - Update post
- `DELETE /posts/{post_id}` - Delete post
- `GET /posts/{post_id}/comments` - Get post comments
- `POST /posts/{post_id}/comments` - Add comment to post
- `GET /authors/` - Get all authors
- `POST /authors/` - Create new author

## Exercise 3: Advanced Features
Extend the blog API with:
- Search functionality (by title, author, content)
- Filtering (by publication date, author, published status)
- Authentication simulation (using dependency injection)
- Error handling for all edge cases
- Background task for sending "new post" notifications

## Challenge: E-commerce API
Create a mini e-commerce API with products, orders, and customers:
- Implement proper relationships between models
- Add inventory management
- Include order status tracking
- Add product search and filtering
- Implement proper error handling and validation

# 🏡Տնային

## Homework 1: Library Management System (Pydantic Focus)
Create Pydantic models for a library management system:

### Required Models:
1. **Member**
   - member_id, name, email, phone, address, membership_date
   - Add validation for email and phone formats
   - Add age calculation from birth_date

2. **Book** 
   - book_id, title, authors (list), isbn, genre, publication_year, pages
   - Custom validator for ISBN format
   - Ensure publication_year is reasonable

3. **BorrowRecord**
   - record_id, member_id, book_id, borrow_date, due_date, return_date
   - Automatically calculate due_date (14 days from borrow_date)
   - Validate that return_date is not before borrow_date

### Requirements:
- Use proper Field constraints
- Add comprehensive custom validators
- Include proper error handling examples
- Create test data and demonstrate validation

## Homework 2: Task Management API (FastAPI Focus)  
Build a complete task management REST API:

### Features Required:
1. **User Management**
   - Register, login (simulation), get profile
   - Users can only access their own tasks

2. **Task Management**
   - CRUD operations for tasks
   - Tasks have: title, description, status, priority, due_date, assigned_user
   - Filter tasks by status, priority, due date
   - Search tasks by title/description

3. **Advanced Features**
   - Task status transitions (todo → in_progress → done)
   - Bulk operations (mark multiple tasks as done)
   - Task statistics endpoint
   - Export tasks to JSON

### Technical Requirements:
- Use proper HTTP status codes
- Implement comprehensive error handling
- Add request/response logging
- Include API documentation
- Write unit tests for all endpoints
- Use dependency injection for user authentication

## Homework 3: Integration Project
Combine both technologies to create a **Recipe Sharing Platform**:

### Models (Pydantic):
- User, Recipe, Ingredient, Review, Category
- Complex validation rules
- Nested structures for ingredients with quantities

### API (FastAPI):
- Complete CRUD for all entities
- Recipe search and filtering
- User authentication simulation
- Recipe rating system
- Favorite recipes functionality
- Recipe recommendation endpoint

### Deliverables:
1. Working FastAPI application
2. Comprehensive test suite
3. API documentation (auto-generated + manual)
4. Example client code showing API usage
5. Performance considerations document

### Bonus Points:
- Implement caching for frequently accessed recipes
- Add image upload simulation for recipes
- Create a simple HTML frontend
- Add rate limiting for API endpoints
- Implement soft delete functionality

# 🎲 00
- ▶️[Video]()
- 🔗[Random link]()
- 🇦🇲🎶[]()
- 🌐🎶[]()
- 🤌[Կարգին]()


<a href="http://s01.flagcounter.com/more/1oO"><img src="https://s01.flagcounter.com/count2/1oO/bg_FFFFFF/txt_000000/border_CCCCCC/columns_2/maxflags_10/viewers_0/labels_0/pageviews_1/flags_0/percent_0/" alt="Flag Counter"></a>
